In [46]:
import torch
from torch_geometric.data import Data
from torch_geometric import seed_everything
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
import torch_geometric
from tqdm import tqdm, trange
from torcheval.metrics import MulticlassAccuracy
from torcheval.metrics import BinaryAUROC
import numpy as np

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE = torch.device('cuda')
print(DEVICE)

cuda


In [47]:
# get feature from csv
RAW_data = pd.read_csv('data/adult.csv')
CAT = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']
NUM = ['age','fnlwgt','educational-num','capital-gain','capital-loss','hours-per-week']
LABEL = 'income'
# RAW_data = pd.read_csv('data/compass_old.csv')
# CAT=['sex','age_cat','race','c_charge_degree','decile_score.1','score_text','v_type_of_assessment','v_decile_score','v_score_text']
# NUM=['age','juv_fel_count','juv_misd_count','juv_other_count','priors_count','days_b_screening_arrest','c_days_from_compas','end']
# LABEL = 'is_recid'
# convert categorical data to ordinal data
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
data_pd = RAW_data.copy()
data_pd[CAT] = enc.fit_transform(RAW_data[CAT])
# data_pd = pd.get_dummies(RAW_data, columns=CAT, dtype=float)
# label to category
data_pd[LABEL] = data_pd[LABEL].astype('category').cat.codes

# realign data to num + cat
data_pd = data_pd[NUM + CAT + [LABEL]]

# caculate unique value of each categorical feature
cat_num = [len(data_pd[col].unique()) for col in CAT]

# normalize numerical data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_pd[NUM] = scaler.fit_transform(data_pd[NUM])

# convert data to tensor
x = torch.tensor(data_pd.drop(columns=[LABEL]).values, dtype=torch.float, device=DEVICE)  # [48842, 108]
y = torch.tensor(data_pd[LABEL].values, dtype=torch.long, device=DEVICE) # [48842]
print(x.shape, y.shape)
print(cat_num)
data_pd

torch.Size([48842, 14]) torch.Size([48842])
[9, 16, 7, 15, 6, 5, 2, 42]


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,gender,native-country,income
0,-0.995129,0.351675,-1.197259,-0.144804,-0.217127,-0.034087,4.0,1.0,4.0,7.0,3.0,2.0,1.0,39.0,0
1,-0.046942,-0.945524,-0.419335,-0.144804,-0.217127,0.772930,4.0,11.0,2.0,5.0,0.0,4.0,1.0,39.0,0
2,-0.776316,1.394723,0.747550,-0.144804,-0.217127,-0.034087,2.0,7.0,2.0,11.0,0.0,4.0,1.0,39.0,1
3,0.390683,-0.277844,-0.030373,0.886874,-0.217127,-0.034087,4.0,15.0,2.0,7.0,0.0,2.0,1.0,39.0,1
4,-1.505691,-0.815954,-0.030373,-0.144804,-0.217127,-0.841104,0.0,15.0,4.0,0.0,3.0,4.0,0.0,39.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,-0.849254,0.640492,0.747550,-0.144804,-0.217127,-0.195490,4.0,7.0,2.0,13.0,5.0,4.0,0.0,39.0,0
48838,0.098933,-0.334178,-0.419335,-0.144804,-0.217127,-0.034087,4.0,11.0,2.0,7.0,0.0,4.0,1.0,39.0,1
48839,1.411808,-0.357510,-0.419335,-0.144804,-0.217127,-0.034087,4.0,11.0,6.0,1.0,4.0,4.0,0.0,39.0,0
48840,-1.213941,0.111984,-0.419335,-0.144804,-0.217127,-1.648120,4.0,11.0,4.0,1.0,3.0,4.0,1.0,39.0,0


In [48]:
class feature_improtance_extractor():
    def __init__(self):
        self.feature_importance = []
        self.iter = 0
        pass
    def update(self, new_feature_importance):
        if self.iter == 0:
            self.feature_importance = new_feature_importance
        else:
            self.feature_importance += new_feature_importance
        self.iter += 1
        return
    
    def get(self):
        return (self.feature_importance / self.iter)
    
    def reset(self):
        self.feature_importance = []
        self.iter = 0
        return
extractor = feature_improtance_extractor()

In [49]:
class K_graph(torch.nn.Module):
    def __init__(self, NUM, CAT, LABEL, cat_num):
        super(K_graph, self).__init__()
        '''
        num_cols: number of numerical columns
        cat_cols: number of categorical columns
        label_cols: number of label columns
        cat_num: number of unique value of each categorical columns
        '''
        self.hidden_dim = 128
        # order: num -> cat -> label
        self.num_cols = len(NUM)
        self.cat_cols = len(CAT)
        self.label_cols = len(LABEL)
        self.number_of_columns = self.num_cols + self.cat_cols 
        self.K = round(self.number_of_columns*0.3)
        
        # numerical feature
        self.num_embeddings = torch.nn.ModuleList([torch.nn.Linear(1, self.hidden_dim) for i in range(self.num_cols)])
        # categorical feature
        self.cat_embeddings = torch.nn.ModuleList([torch.nn.Embedding(cat_num[i], self.hidden_dim) for i in range(self.cat_cols)])
        
        self.prediction = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim *( self.K + self.number_of_columns), self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(self.hidden_dim),
            torch.nn.Linear(self.hidden_dim, self.label_cols + 1)
        )
        
        # feature importance learning
        self.feature_importance_learners = torch.nn.ModuleList([torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(self.hidden_dim),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(self.hidden_dim, 1),
        ) for i in range(self.number_of_columns)])
        
        for i in range(self.number_of_columns):
            torch.nn.init.constant_(self.feature_importance_learners[i][0].weight, 0)
            torch.nn.init.constant_(self.feature_importance_learners[i][0].bias, 0)
            torch.nn.init.constant_(self.feature_importance_learners[i][4].weight, 0)
            torch.nn.init.constant_(self.feature_importance_learners[i][4].bias, 0)
        
        # graph convolution layers
        # self.conv_GCN_input = torch_geometric.nn.GCNConv(self.number_of_columns*self.hidden_dim, self.hidden_dim)
        self.conv_GCN_input = torch_geometric.nn.GCNConv(self.hidden_dim, self.hidden_dim)
        # self.conv_1_input = torch_geometric.nn.GATConv(self.number_of_columns*self.hidden_dim, self.hidden_dim)
        self.conv_GCN_2 = torch_geometric.nn.GCNConv(self.hidden_dim, self.hidden_dim)
        
        # self.transform = torch.nn.Linear(self.number_of_columns*self.hidden_dim, self.hidden_dim)
        
    def forward(self, input_data, epoch = -1):
        
        # make feature embedding
        num_data = input_data[:,:self.num_cols].unsqueeze(-1).unsqueeze(-1) 
        feature_embedding_num = torch.cat([self.num_embeddings[i](num_data[:,i]) for i in range(self.num_cols)], dim=1).reshape(len(input_data), -1) # [batch_size, num_cols * hidden_dim]
        feature_embedding_num = torch.nn.ReLU()(feature_embedding_num)
        feature_embedding_num = torch.layer_norm(feature_embedding_num, feature_embedding_num.shape)
        # categorical feature
        feature_embedding_cat = torch.cat([self.cat_embeddings[i](input_data[:,self.num_cols+i].long()) for i in range(self.cat_cols)], dim=1) # [batch_size, cat_cols * hidden_dim]
        feature_embedding_cat = torch.layer_norm(feature_embedding_cat, feature_embedding_cat.shape)
        # concat
        feature_embedding = torch.cat((feature_embedding_num, feature_embedding_cat), dim=1) # [batch_size, (num_cols + cat_cols) * hidden_dim]
        # feature_embedding = feature_embedding.reshape((len(input_data), self.number_of_columns, -1)) # [batch_size, (num_cols + cat_cols), hidden_dim]
        
        # feature importance learning
        feature_importance = torch.cat([self.feature_importance_learners[i](feature_embedding[:,i*self.hidden_dim:(i+1)*self.hidden_dim]) for i in range(self.number_of_columns)], dim=1) # [batch_size, num_cols + cat_cols, 1]
        feature_importance = torch.layer_norm(feature_importance, feature_importance.shape)
        feature_importance = torch.softmax(feature_importance, dim=1) # [batch_size, num_cols + cat_cols, 1]
        # print(feature_importance.shape)
        # print(feature_importance.sum(dim=1))
        # print(feature_importance)
        
        # weighted feature embedding 
        feature_embedding = feature_embedding.reshape((len(input_data),self.number_of_columns, -1)) * feature_importance.unsqueeze(-1) # [batch_size, (num_cols + cat_cols) * hidden_dim]
        feature_embedding = feature_embedding.reshape((len(input_data), -1)) # [batch_size, (num_cols + cat_cols) * hidden_dim]
        
        # top K feature importance
        K = self.K
        value, indices = torch.topk(feature_importance, K) # (value: [batch_size, k], indices: [batch_size, k])
        mask = torch.zeros_like(feature_importance, device=DEVICE)
        mask.scatter_(1, indices, 1)
        importance_topK = torch.where(mask > 0, feature_importance, torch.zeros(feature_importance.shape,device=DEVICE)) # [batch_size, cols]
        # importance_topK = torch.stack([importance_topK.clone() for _ in range(self.number_of_columns)], dim=0) # [cols, batch_size, cols]
        
        extractor.update(feature_importance.sum(dim=0))
        del feature_embedding_num, feature_embedding_cat, num_data
        del mask, feature_importance, value, indices
        
        
        processed_data = []
        processed_indices = []
        for target_col in range(self.number_of_columns):
            importance_topK_current = importance_topK.clone()# [batch_size, cols] 
            indices = importance_topK_current.T[target_col].nonzero().T[0] # selected samples' indices  
            
            if indices.shape[0] == 0:
                continue
            
            importance_topK_current = importance_topK_current[importance_topK_current.T[target_col]>0]# [????, cols]
            
            # for target column, set its importance to 0. so that it will not be fully connected graph
            # copy target column
            tmp = torch.clone(importance_topK_current[:,target_col]) # [????], save for future weighted sum
            importance_topK_current[:,target_col] = 0 # [????, cols]
            # multiply to get weighted adj
            weighted_adj = torch.matmul(importance_topK_current, importance_topK_current.T) # [batch_size, cols] * [cols, batch_size] = [batch_size, batch_size]
            # prune the diagonal
            weighted_adj = weighted_adj - torch.diag(weighted_adj.diagonal())

            # construct graph
            edge_index = weighted_adj.nonzero().T  # [2, num_edges]
            edge_wight = weighted_adj[edge_index[0], edge_index[1]] # [num_edges]
            edge_wight = torch.softmax(edge_wight, dim=0)

            
            if False:
                print('in graph', target_col, 'nodes:', len(indices), 'edges:', len(edge_wight),'ratio', len(edge_wight)/(len(indices)**2+0.000001))
            
            # print(edge_wight)
            # importance_topK_current[:,target_col] = tmp # [????, cols]
            
            # features = (feature_embedding[indices]) # [????, cols*hidden_dim]
            features = (feature_embedding.reshape(len(input_data),self.number_of_columns,-1)[indices][:,target_col,:]) # [????, hidden_dim]
            # print(features.shape)

            # construct graph 
            data = Data(x=features, edge_index=edge_index, edge_weight=edge_wight, indices=indices) 
            
            del features, edge_index, edge_wight, weighted_adj, importance_topK_current, tmp
            
            # apply GCN
            x = self.conv_GCN_input(data.x, data.edge_index, data.edge_weight)  # [???, hidden_dim]
            # x = self.conv_1_input(data.x, data.edge_index)  # [???, hidden_dim]
            x = torch.relu(x)
            x = torch.layer_norm(x, x.shape) # [???, hidden_dim]
            x = torch.nn.Dropout(p=0.5)(x)
            x = self.conv_GCN_2(x, data.edge_index, data.edge_weight)  # [???, hidden_dim]
            x = torch.relu(x)
            x = torch.layer_norm(x, x.shape)

            processed_data.append(x)
            processed_indices.append(indices)
        
        processed_data = torch.cat(processed_data, dim=0) 
        processed_indices = torch.cat(processed_indices, dim=0)
        # print(processed_indices)
        # print(processed_indices.argsort())
        # print(processed_indices[processed_indices.argsort()])
        processed_data = processed_data[processed_indices.argsort()] # [batch_size, hidden_dim]
        processed_data = torch.split(processed_data, self.K) # (batch_size, K, hidden_dim)
        processed_data = torch.stack(list(processed_data), dim=0) # [batch_size, K, hidden_dim]

        # cat residual
        processed_data = torch.cat((processed_data, feature_embedding.reshape((len(input_data),self.number_of_columns,-1))), dim=1) # [batch_size, K+cols , hidden_dim]
        
        # make prediction
        prediction = self.prediction(processed_data.reshape(processed_data.shape[0],-1))
        # prediction = self.prediction(feature_embedding)
        
        
        return prediction


In [50]:
the_model = K_graph(NUM, CAT, [LABEL], cat_num).to(DEVICE)
optimizer = torch.optim.SGD(the_model.parameters(), lr=0.001)

# optimizer.step()
data_count = 5
# random pick data
indices = torch.randperm(len(x))[:data_count]
train_data = x[indices]
train_label = y[indices]

for i in range(2):
    
    optimizer.zero_grad()
    output = the_model(train_data[:data_count], epoch=200)
    loss = torch.nn.functional.cross_entropy(output, train_label[:data_count])
    loss.backward()
    # print(((the_model.feature_importance_learners.grad).abs().max(dim=1)[0]))
    optimizer.step()
    
    print('-----------------------------------------')


-----------------------------------------
-----------------------------------------


In [51]:
import torchviz
plot = torchviz.make_dot(loss, params=dict(the_model.named_parameters()))
plot.render("MLP+FI", format="png")

'MLP+FI.png'

In [52]:

def train_epoch(model, optimizer, datas, batch_size, epoch):
    train_data, train_label, validation_data, validation_label = datas
    
    # slice data into batch
    train_data = torch.split(train_data, batch_size)
    train_label = torch.split(train_label, batch_size)
    validation_data = torch.split(validation_data, batch_size)
    validation_label = torch.split(validation_label, batch_size)

    # losses and metrics
    batch_loss = 0
    train_acc = MulticlassAccuracy(num_classes=2).to(DEVICE)
    train_auc = BinaryAUROC().to(DEVICE)
    valid_acc = MulticlassAccuracy(num_classes=2).to(DEVICE)
    valid_auc = BinaryAUROC().to(DEVICE)
    
    # train the model
    stepper = trange(len(train_data))
    for i in stepper:
        stepper.set_description(f'Epoch {epoch}')
        
        optimizer.zero_grad()
        output = model(train_data[i], epoch=epoch)
        loss = torch.nn.functional.cross_entropy(output, train_label[i]) * model.number_of_columns
        loss.backward()
        optimizer.step()
        batch_loss += loss.item()
        
        # metrics
        preds = output.softmax(dim=1)
        true = torch.nn.functional.one_hot(train_label[i], num_classes=2).to(DEVICE)
        train_acc.update(torch.argmax(preds, 1),true.T[1])
        train_auc.update(preds.T[0],true.T[0])
        
        # at the end of epoch, print result and validate the model
        if i == len(train_data) - 1:
            train_acc = train_acc.compute()
            train_auc = train_auc.compute()
            stepper.set_postfix({'loss': round(batch_loss/(i+1), 3), 'acc': round(train_acc.item(), 3), 'AUC': round(train_auc.item(), 3)})
            stepper.update()
        
            with torch.no_grad():
                for i in range(len(validation_data)):
                    output = model(validation_data[i], epoch=200)
                    # loss = torch.nn.functional.cross_entropy(output, validation_label[i])
                    preds = output.softmax(dim=1)
                    true = torch.nn.functional.one_hot(validation_label[i], num_classes=2).to(DEVICE)
                    valid_acc.update(torch.argmax(preds,1),true.T[1])
                    valid_auc.update(preds.T[0],true.T[0])
                stepper.set_postfix({'loss': round(batch_loss/(i+1), 3), 'acc': round(train_acc.item(), 3), 'AUC': round(train_auc.item(), 3), 'val_acc': round(valid_acc.compute().item(), 3), 'val_AUC': round(valid_auc.compute().item(), 3)})



In [53]:
def overall_train(x, y, seed=0):
    # hyperparameter
    epoch = 50
    batch_size = 1000
    seed_everything(seed)
    
    # shuffle data
    indices = torch.randperm(len(x))
    x = x[indices]
    y = y[indices]
    # slice data into train and test and validation
    train_ratio = 0.7
    validation_ratio = 0.1
    train_data = x[:int(len(x)*train_ratio)]
    train_label = y[:int(len(x)*train_ratio)]
    validation_data = x[int(len(x)*train_ratio):int(len(x)*(train_ratio+validation_ratio))]
    validation_label = y[int(len(x)*train_ratio):int(len(x)*(train_ratio+validation_ratio))]
    test_data = x[int(len(x)*(train_ratio+validation_ratio)):]
    test_label = y[int(len(x)*(train_ratio+validation_ratio)):]

    # build model and optimizer
    the_model = K_graph(NUM, CAT, [LABEL], cat_num).to(DEVICE)
    optimizer = torch.optim.SGD(the_model.parameters(), lr=0.001)
    
    # train the model
    datas = (train_data, train_label, validation_data, validation_label)
    for i in range(epoch):
        train_epoch(the_model, optimizer, datas, batch_size, epoch=i+1)
        print(extractor.get())
        extractor.reset()
    
    # test the model
    with torch.no_grad():
        test_data = torch.split(test_data, batch_size)
        test_label = torch.split(test_label, batch_size)
        for i in range(len(test_data)):
            output = the_model(test_data[i], epoch=200)
            preds = output.softmax(dim=1)
            true = torch.nn.functional.one_hot(test_label[i], num_classes=2).to(DEVICE)
            test_acc = MulticlassAccuracy(num_classes=2).to(DEVICE)
            test_auc = BinaryAUROC().to(DEVICE)
            test_acc.update(torch.argmax(preds,1),true.T[1])
            test_auc.update(preds.T[0],true.T[0])
        # output = the_model(test_data, epoch=200)
        # preds = output.softmax(dim=1)
        # true = torch.nn.functional.one_hot(test_label, num_classes=2).to(DEVICE)
        # test_acc = MulticlassAccuracy(num_classes=2).to(DEVICE)
        # test_auc = BinaryAUROC().to(DEVICE)
        # test_acc.update(torch.argmax(preds,1),true.T[1])
        # test_auc.update(preds.T[0],true.T[0])
        print('test_acc:', test_acc.compute().item())
        print('test_auc:', test_auc.compute().item())
        print('-----------------------------------------')

In [54]:
seed_set = [9,90,900,9000,90000]
overall_train(x, y, seed=9)

Epoch 1: 100%|██████████| 35/35 [00:03<00:00, 10.32it/s, loss=44.3, acc=0.782, AUC=0.796, val_acc=0.806, val_AUC=0.844]


tensor([ 42.2897,  18.1787,  77.1797,  32.8793,  31.9462,  57.2528,  10.9467,
         93.0086,  12.9357,  25.0136, 341.2386,  31.2167,  12.8721, 143.5891],
       device='cuda:0', grad_fn=<DivBackward0>)


Epoch 2: 100%|██████████| 35/35 [00:03<00:00, 11.58it/s, loss=39.4, acc=0.807, AUC=0.843, val_acc=0.804, val_AUC=0.846]


tensor([ 47.1663,  15.9831,  57.2612,  44.8532,  31.1761,  54.6521,   9.4843,
         64.7513,  11.2442,  24.1347, 491.0917,  26.4991,  11.4132,  87.1145],
       device='cuda:0', grad_fn=<DivBackward0>)


Epoch 3: 100%|██████████| 35/35 [00:03<00:00, 11.50it/s, loss=38.7, acc=0.809, AUC=0.851, val_acc=0.817, val_AUC=0.873]


...

tensor([128.4901,  16.6742, 121.6351, 131.7854,  62.6056, 124.1348,  20.8413,
         21.9341,  12.4470, 133.6166, 144.7003,  12.4713,  32.3671,  13.1220],
       device='cuda:0', grad_fn=<DivBackward0>)


Epoch 49: 100%|██████████| 35/35 [00:03<00:00, 11.43it/s, loss=30.7, acc=0.854, AUC=0.909, val_acc=0.843, val_AUC=0.905]


tensor([129.7317,  16.7055, 117.7386, 133.4590,  60.9776, 129.0655,  21.1516,
         21.8458,  12.3668, 131.4819, 143.3666,  12.3793,  33.5186,  13.0365],
       device='cuda:0', grad_fn=<DivBackward0>)


Epoch 50: 100%|██████████| 35/35 [00:03<00:00, 11.41it/s, loss=30.4, acc=0.855, AUC=0.911, val_acc=0.847, val_AUC=0.907]


tensor([130.3251,  16.6935, 119.9304, 135.4450,  61.6927, 125.5841,  21.5125,
         21.8529,  12.2738, 133.7882, 138.0812,  12.2931,  34.4315,  12.9213],
       device='cuda:0', grad_fn=<DivBackward0>)
test_acc: 0.8621586561203003
test_auc: 0.9137257112789867
-----------------------------------------


In [55]:
sum([0.6960486173629761,
     0.6990881562232971,
     0.7203647494316101,
     0.7355623245239258,
     0.7446808218955994
     ])/5

0.7191489338874817

In [56]:
sum([0.765040650406504,
     0.7699334319526627,
     0.7728218852215284,
     0.7938576945929887,
     0.8200266134397871
     ])/5

0.7843360551226942